# Look for mutations that improve binding of BtKy72 RBD to mouse but **not** human ACE2

In [1]:
import altair as alt

import pandas as pd

Read mutation effects:

In [2]:
muteffects = (
    pd.read_csv("../results/final_variant_scores/mut_variant_scores.csv")
    .query("target == 'BtKY72'")
    [
        [
            "wildtype",
            "position",
            "mutant",
            "huACE2",
            "mACE2",
        ]
    ]
    .assign(mutation=lambda x: x["wildtype"] + x["position"].astype(str) + x["mutant"])
)

muteffects

,wildtype,position,mutant,huACE2,mACE2,mutation
240,L,455,A,5.000,5.0257,L455A
241,L,455,C,5.000,5.0000,L455C
242,L,455,D,5.000,5.0004,L455D
243,L,455,E,5.000,5.0528,L455E
244,L,455,F,5.000,5.0629,L455F
...,...,...,...,...,...,...
355,V,501,S,5.000,5.0200,V501S
356,V,501,T,5.000,5.0074,V501T
357,V,501,V,5.000,5.0341,V501V
358,V,501,W,5.000,5.0000,V501W


Make a scatter chart:

In [3]:
mutation_selector = alt.selection_single(on="mouseover", empty="none", fields=["mutation"])

tooltips = [
    "mutation", alt.Tooltip("huACE2", format=".3f"), alt.Tooltip("mACE2", format=".3f"),
]

scatter = (
    alt.Chart(muteffects)
    .encode(
        x=alt.X("huACE2", scale=alt.Scale(zero=False), axis=alt.Axis(grid=False)),
        y=alt.Y("mACE2", scale=alt.Scale(zero=False), axis=alt.Axis(grid=False)),
        tooltip=tooltips,
        strokeWidth=alt.condition(mutation_selector, alt.value(2), alt.value(0)),
        size=alt.condition(mutation_selector, alt.value(100), alt.value(50)),
        opacity=alt.condition(mutation_selector, alt.value(1), alt.value(0.6)),
    )
    .mark_circle(stroke="black")
    .properties(width=200, height=200)
    .add_selection(mutation_selector)
)

scatter

alt.Chart(...)

Make heatmaps:

In [4]:
muteffects_tidy = muteffects.melt(
    id_vars=["mutation", "wildtype", "position", "mutant"],
    value_vars=["huACE2", "mACE2"],
    var_name="ACE2",
    value_name="affinity",
).merge(muteffects[["mutation", "huACE2", "mACE2"]])

In [5]:
heatmap_base = (
    alt.Chart(muteffects_tidy)
    .encode(
        x=alt.X("position:O"),
        y=alt.Y("mutant", title="amino acid"),
        tooltip=tooltips,
    )
    .properties(height=alt.Step(12), width=alt.Step(12))
)

heatmap_colors = (
    heatmap_base
    .encode(
        color=alt.Color("affinity"),
        strokeWidth=alt.condition(mutation_selector, alt.value(3), alt.value(1)),
    )
    .mark_rect(stroke="black")
)

heatmap_wildtype = (
    heatmap_base
    .transform_filter(alt.datum.wildtype == alt.datum.mutant)
    .mark_text(text="x", color="black")
)

heatmap = (
    (heatmap_colors + heatmap_wildtype)
    .facet(
        column=alt.Column(
            "ACE2",
            title=None,
            header=alt.Header(labelFontWeight="bold", labelFontSize=12),
        ),
    )
    .add_selection(mutation_selector)
)

heatmap

alt.FacetChart(...)

Make a merged chart:

In [6]:
merged_chart = scatter | heatmap

merged_chart.save("BtKY72-mouse-ACE2-only.html")

merged_chart

alt.HConcatChart(...)